# Máster en Data Science - Machine Learning

# Exploratory Data Analysis (EDA)

Autores: Frida Ibarra y Gema Romero

#### **Notebook 3 (Data_Encoding_Scaling)** 

Centrado en la codificación y el escalamiento de los datos. La preparación adecuada de los datos es un paso fundamental en el desarrollo de modelos predictivos, ya que garantiza que todas las variables sean compatibles con los algoritmos utilizados. Este proceso incluye:

1. Codificación de variables categóricas. ransformación de variables categóricas en formatos numéricos, asegurando que los algoritmos puedan interpretar correctamente la información. Se seleccionará el método de codificación más adecuado según las características de las variables (nominales u ordinales) y el número de categorías.

2. Escalamiento de los datos. Normalización o estandarización de las variables numéricas para garantizar que todas operen en una misma escala, reduciendo posibles sesgos e incrementando la eficiencia del modelo.

In [1]:
# Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
import warnings
import sys
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [10]:
#FUNCIONES
sys.path.append('../src')  # Asegúrate de que ../src es la carpeta donde está Funciones_Ayuda.py
import functions_src as fa  # Ahora debería importarse correctamente
sys.path.remove('../src')

#Semilla 
seed = 25

In [11]:
df_loans_train = pd.read_csv('../data/Processing_data/df_loans_train.csv')
df_loans_test = pd.read_csv('../data/Processing_data/df_loans_test.csv')
df_loans_train.head()

,Unnamed: 0,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
0,84056,197497,Cash loans,F,N,N,1,85500.0,862560.0,25348.5,720000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.002134,-11809,-3745,-1335.0,-2426,NaN,1,1,0,1,0,0,Medicine staff,3.0,3,3,WEDNESDAY,8,0,0,0,1,1,1,Medicine,0.110762,0.076364,0.380800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1.0,4.0,1.0,-707.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,2.0,5.0,0
1,195841,327085,Cash loans,M,Y,Y,0,180000.0,1006920.0,42790.5,900000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.003122,-13873,-3460,-2734.0,-5159,14.0,1,1,0,1,0,0,NaN,2.0,3,3,WEDNESDAY,15,0,0,0,0,0,0,Business Entity Type 3,0.270259,0.474984,0.700184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-1425.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0
2,79945,192672,Cash loans,M,N,Y,0,180000.0,238896.0,13842.0,189000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.031329,-16489,-8675,-7422.0,-37,NaN,1,1,0,1,0,0,NaN,2.0,2,2,SUNDAY,12,0,0,0,0,0,0,Industry: type 5,NaN,0.737165,0.621226,0.0124,0.0169,0.9682,0.5648,0.0074,0.0,0.0690,0.0417,0.0417,0.0201,0.0101,0.0097,0.0,0.0,0.0126,0.0175,0.9682,0.5818,0.0074,0.0,0.0690,0.0417,0.0417,0.0206,0.0110,0.0101,0.0,0.0,0.0125,0.0169,0.9682,0.5706,0.0074,0.0,0.0690,0.0417,0.0417,0.0205,0.0103,0.0099,0.0,0.0,reg oper spec account,block of flats,0.0117,"Stone, brick",No,0.0,0.0,0.0,0.0,-1574.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [12]:
# Se va a eliminar la primera columna que repite el Index
df_loans_train = df_loans_train.drop('Unnamed: 0',axis=1)
df_loans_test = df_loans_test.drop('Unnamed: 0',axis=1)

In [5]:
# Diccionario de los datos:
var_description = pd.read_excel('../data/columns_description.xlsx')

### Codificación de Variables Categóricas

Las variables categóricas representan atributos discretos que no pueden ser interpretados como valores numéricos directos. Sin embargo, al aplicar una codificación adecuada, podemos transformarlas en formatos numéricos que los algoritmos de modelado puedan comprender.

Estas variables se dividen en dos tipos: ordinales y nominales. Las variables ordinales tienen un orden o jerarquía entre sus categorías, lo que permite establecer una relación de mayor o menor. En cambio, las variables nominales agrupan elementos en categorías distintas, pero sin que exista un orden o jerarquía entre ellas.

El objetivo, por lo tanto, es convertir estas variables categóricas en representaciones numéricas dentro de nuestro conjunto de datos.

Existen diversas técnicas para realizar esta codificación:

1. **One-Hot Encoding**: Este método transforma cada valor único de una variable categórica en una columna binaria (0 o 1). Se asigna un 1 para la categoría presente y 0 para las demás. Es adecuado cuando las categorías no tienen un orden inherente, evitando que el modelo asuma una relación jerárquica entre ellas.

2. **Ordinal Encoding**: Este enfoque asigna un valor numérico a cada categoría de una variable categórica, respetando el orden natural de las categorías. Se usa cuando las categorías tienen un orden jerárquico.

3. **Mean Encoding**: Consiste en reemplazar cada valor de la variable categórica por la media de la variable objetivo correspondiente a esa categoría. Es útil cuando existe una relación estadística entre la variable categórica y la variable objetivo, pero puede provocar sobreajuste si no se maneja adecuadamente.

4. **Target Encoding**: Similar al mean encoding, pero en lugar de usar la media global, se utiliza el promedio de la variable objetivo para cada categoría específica. Este método incorpora información más relevante para cada categoría, pero debe ser usado con precaución para evitar el sobreajuste.

5. **CatBoostEncoder**: variante de la codificación de objetivo que admite objetivos **binomiales** y **continuos**, y ofrece **codificación consciente del tiempo**, **regularización** y **aprendizaje en línea**. Es sensible al orden de los datos, lo que lo hace ideal para problemas de series temporales. Si los datos no tienen dependencia temporal, funcionará bien siempre que no haya fuga de información, la cual debe evitarse mediante técnicas como **shuffling** o **resampling**.

### Codificación de Variables Categóricas aplicada a esta práctica

Al principio de esta práctica, se pudo comprobar que hay variables de naturaleza categórica en formato string. Esto puede dificultar el análisis de las variables, ya que muchos algoritmos de machine learning requieren que las variables categóricas sean transformadas a un formato numérico. Además, el manejo directo de variables de tipo string puede generar problemas de rendimiento y precisión, sobre todo cuando las categorías tienen muchas variaciones o valores distintos. Para abordar este problema, se van a utilizar algunas de las técnicas de codificación de categorías mencionadas en el apartado anterior, que permiten transformar estas variables de manera eficiente. 

Elegir la técnica adecuada va a depender de la naturaleza de los datos y de la cantidad de categorías. Al aplicar estas técnicas, es posible mejorar la interpretación y la capacidad de los modelos predictivos al trabajar con variables categóricas de manera más eficiente.


In [14]:
# Llamar a la función y guardar los resultados en variables
col_bool, col_cat, col_num = fa.categorizar_columnas(df_loans_train)

In [7]:
cat_vars = df_loans_train.select_dtypes(include=['object']).columns

# Contar valores únicos en cada variable categórica
unique_counts = df_loans_train[cat_vars].nunique().sort_values(ascending=False)

print(unique_counts)

ORGANIZATION_TYPE             58
OCCUPATION_TYPE               18
NAME_INCOME_TYPE               8
NAME_TYPE_SUITE                7
WEEKDAY_APPR_PROCESS_START     7
WALLSMATERIAL_MODE             7
NAME_HOUSING_TYPE              6
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             5
FONDKAPREMONT_MODE             4
CODE_GENDER                    3
HOUSETYPE_MODE                 3
NAME_CONTRACT_TYPE             2
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
EMERGENCYSTATE_MODE            2
dtype: int64


Gracias al resultado del análisis de WOE e IV se ha podido conocer la relación entre la variable target y las variables categóricas. Dentro de las que tienen un impacto moderado (pero no fuerte), se encuentran: *CODE_GENDER, NAME_INCOME_TYPE, NAME_FAMILY_STATUS, OCCUPATION_TYPE*. Por ello, se establecen los siguientes criterios de encoding:

- Baja cardinalidad (≤ 8 categorías): Se usará One-Hot Encoding, ya que es eficiente para variables con pocas categorías y no añade complejidad innecesaria al modelo.  

- Media cardinalidad (9-50 categorías): No se aplicará Target Encoding debido a que las variables en este rango no muestran un impacto significativo en la relación con la variable objetivo, por lo que no justifica su uso. En su lugar: 

- Para la variable OCCUPATION_TYPE (18 categorías), se aplicará Mean Encoding, ya que permite capturar de manera eficiente la relación promedio entre cada categoría y la variable objetivo.

- Alta cardinalidad (> 50 categorías): Se aplicará CatBoost Encoding para la variable ORGANIZATION_TYPE (58 categorías), una técnica adecuada para manejar variables de alta cardinalidad reduciendo el riesgo de overfitting.
Este enfoque busca balancear la complejidad del modelo y la relevancia de las variables categóricas, maximizando el desempeño general del modelo.



#### Separación x e y 

In [8]:
y_train = df_loans_train['TARGET']
X_train = df_loans_train.drop('TARGET', axis=1)
y_test = df_loans_test['TARGET']
X_test = df_loans_test.drop('TARGET', axis=1)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((246008, 121), (246008,), (61503, 121), (61503,))

#### CODIFICACIÓN ONE HOT ENCODING

In [ ]:
list_columns_cat = list(df_loans_train.select_dtypes(include=["object", "category"]).columns)
exclude_vars = ['OCCUPATION_TYPE', 'ORGANIZATION_TYPE']  # Excluir estas columnas
list_columns_ohe = [col for col in list_columns_cat if col not in exclude_vars]

# Crear y aplicar One-Hot Encoder
ohe = ce.OneHotEncoder(cols=list_columns_ohe, use_cat_names=True)
ohe.fit(X_train, y_train)  

# Transformar X_train y X_test
X_train_t = ohe.transform(X_train)
X_test_t = ohe.transform(X_test)

# Verificar formas finales
print(X_train_t.shape, X_test_t.shape)

(246008, 175) (61503, 175)


#### CODIFICACIÓN MEAN ENCODING

Como se mencionó anteriormente, la variable **OCCUPATION_TYPE** muestra una relación moderada con la variable objetivo (**Target**). Sin embargo, esta relación no es lo suficientemente significativa como para aplicar una codificación basada en el **Target Encoding**. Por ello, se optará por utilizar la técnica de **Mean Encoding** para su tratamiento.

In [ ]:
# Mean Encoding para OCCUPATION_TYPE
target_column = 'OCCUPATION_TYPE'

# Calcular el promedio de 'TARGET' por cada categoría de 'OCCUPATION_TYPE'
mean_encoding_map = X_train_t.copy()
mean_encoding_map['TARGET'] = y_train  # Añadimos la variable TARGET al DataFrame
mean_encoding_map = mean_encoding_map.groupby(target_column)['TARGET'].mean().to_dict()

# Aplicar Mean Encoding a X_train y X_test
X_train_me = X_train_t.copy()
X_test_me = X_test_t.copy()

X_train_me[target_column] = X_train_t[target_column].map(mean_encoding_map)
X_test_me[target_column] = X_test_t[target_column].map(mean_encoding_map).fillna(y_train.mean())

# Verificar formas finales
print(X_train_me.shape, X_test_me.shape)

(246008, 175) (61503, 175)


#### CODIFICACIÓN CATBOOST

In [ ]:
# CatBoost Encoding
target_column = 'ORGANIZATION_TYPE'

# Crear y ajustar el codificador de CatBoost Encoding
catboost_enc = ce.CatBoostEncoder(cols=[target_column])
catboost_enc.fit(X_train_me[target_column], y_train)  

# Transformar X_train y X_test
X_train_mec = X_train_me.copy()
X_test_mec = X_test_me.copy()

X_train_mec[target_column] = catboost_enc.transform(X_train_me[target_column])
X_test_mec[target_column] = catboost_enc.transform(X_test_me[target_column])

# Verificar formas finales
print(X_train_mec.shape, X_test_mec.shape)

(246008, 175) (61503, 175)


Tras la codificación de variables categóricas, ha quedado un dataset de train con 175 columnnas y 246008 instancias y un dataset de test con 175 columnnas y 61503 instancias

#### ESCALARIZACIÓN DE LAS VARIABLES CATEGÓRICAS

Cuando aplicas codificación *one-hot* a tus variables categóricas, estas se transforman en un conjunto de variables binarias cuyos valores son exclusivamente 0 y 1. Este enfoque evita que las variables categóricas influyan negativamente en el modelo al eliminar cualquier noción implícita de orden o jerarquía en los datos originales. Además, esta transformación mejora la compatibilidad con los algoritmos de aprendizaje automático, permitiendo que estos aprovechen de manera más efectiva la información contenida en las categorías para mejorar la calidad de las predicciones.

In [ ]:
#Comprobamos que están todas codificadas
X_train_mec.dtypes.to_dict()


{'SK_ID_CURR': dtype('int64'),
 'NAME_CONTRACT_TYPE_Cash loans': dtype('int64'),
 'NAME_CONTRACT_TYPE_Revolving loans': dtype('int64'),
 'CODE_GENDER_F': dtype('int64'),
 'CODE_GENDER_M': dtype('int64'),
 'CODE_GENDER_XNA': dtype('int64'),
 'FLAG_OWN_CAR_N': dtype('int64'),
 'FLAG_OWN_CAR_Y': dtype('int64'),
 'FLAG_OWN_REALTY_N': dtype('int64'),
 'FLAG_OWN_REALTY_Y': dtype('int64'),
 'CNT_CHILDREN': dtype('int64'),
 'AMT_INCOME_TOTAL': dtype('float64'),
 'AMT_CREDIT': dtype('float64'),
 'AMT_ANNUITY': dtype('float64'),
 'AMT_GOODS_PRICE': dtype('float64'),
 'NAME_TYPE_SUITE_Unaccompanied': dtype('int64'),
 'NAME_TYPE_SUITE_Family': dtype('int64'),
 'NAME_TYPE_SUITE_Spouse, partner': dtype('int64'),
 'NAME_TYPE_SUITE_Children': dtype('int64'),
 'NAME_TYPE_SUITE_Other_A': dtype('int64'),
 'NAME_TYPE_SUITE_Other_B': dtype('int64'),
 'NAME_TYPE_SUITE_Group of people': dtype('int64'),
 'NAME_TYPE_SUITE_nan': dtype('int64'),
 'NAME_INCOME_TYPE_Working': dtype('int64'),
 'NAME_INCOME_TYPE_Pen

In [ ]:
scaler = StandardScaler()
model_scaled = scaler.fit(X_train_mec)
X_train_scaled = pd.DataFrame(scaler.transform(X_train_mec), columns=X_train_mec.columns, index=X_train_mec.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_mec), columns=X_test_mec.columns, index=X_test_mec.index)

In [ ]:
X_train_scaled.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE_Unaccompanied,NAME_TYPE_SUITE_Family,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_nan,NAME_INCOME_TYPE_Working,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Student,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Academic degree,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Widow,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_With parents,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Co-op apartment,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE_reg oper spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_nan,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_terraced house,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_nan,TOTALAREA_MODE,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Panel,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,-0.786989,0.324103,-0.324103,0.722057,-0.722031,-0.004032,

### CONCLUSIONES FINALES DEL EDA

El análisis exploratorio de datos (EDA) ha permitido identificar patrones clave, relaciones significativas y posibles inconsistencias en los datos, proporcionando una base sólida para la preparación y modelado. Este proceso ha sido esencial para comprender la estructura de las variables y su impacto en la variable objetivo, lo que orientará las siguientes etapas del análisis. 

Se han sacado la siguientes conclusiones:

- Exploración y análisis inicial (EDA): Comenzamos con un análisis exploratorio para comprender el tamaño y las características de nuestro conjunto de datos, prestando especial atención a la variable objetivo, TARGET, y sus relaciones con las demás variables.

- Análisis de tipos de variables: Analizamos las variables en términos de su tipo (categóricas, numéricas, etc.) y evaluamos cómo se relacionan con la variable objetivo. Esto nos ayudó a determinar qué técnicas de codificación eran más adecuadas para cada tipo de variable.

- Análisis de correlación: Estudiamos las relaciones de correlación entre las variables para identificar posibles redundancias y relaciones lineales importantes que podrían influir en la selección de características.

- División en conjuntos de entrenamiento y prueba: Realizamos una división estratificada de los datos en conjuntos de entrenamiento y prueba, asegurando que la distribución de la variable objetivo se mantuviera en ambas particiones.

- Tratamiento de outliers: Detectamos y tratamos los valores atípicos en las variables numéricas, asegurándonos de que no afectaran negativamente al modelo.

- Imputación de valores faltantes: Abordamos los valores nulos en el conjunto de datos mediante técnicas de imputación adecuadas, garantizando que las variables estuvieran completas para el entrenamiento del modelo.

- Análisis WoE e IV: Aplicamos el análisis de Weight of Evidence (WoE) y la Información de Valor (IV) a las variables categóricas para medir su capacidad predictiva y determinar qué variables serían más útiles para el modelado.

- Codificación de variables categóricas: Utilizamos técnicas de codificación numérica como One-Hot Encoding, Mean Encoding y CatBoost Encoding para transformar las variables categóricas en valores numéricos adecuados para los modelos.

- Escalado de variables: Procedimos con el escalado de las variables numéricas para asegurar que todas las características tuvieran un rango similar y fueran adecuadamente procesadas por los modelos de Machine Learning.

- Con estos pasos completados, nuestros datos están listos para continuar con las siguientes fases del proyecto: la selección de características (feature processing), la construcción y evaluación de modelos, la implementación del modelo final, y la explicación y conclusiones del trabajo realizado.